# Homework 2

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [70]:

import math
import pandas as pd
import numpy as np
from scipy.stats import cauchy
from scipy.spatial import distance
import plotly.graph_objects as go

# CLASSES

In [71]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

**Activity 1:** Path length - (BM1 vs BM2 vs CRW) (4 pts)
- Write a function that returns a **Brownian Motion (BM)** trajectory in **pandas** df.
- Write a function that returns a **Correlated Random Walk (CRW)** trajectory in **pandas** df.
- Write a function that returns the **path length** for a given trajectory.
- Compare at least the **path length** of **three** trajectories as shown in the figure below.
- Display the results using **plotly**.

**FUNCTIONS**

In [72]:
#####################################################################################
# Brownian motion trajectoy
#####################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
    """
    Arguments:
        n_steps: Number of steps
        speed: Speed of the particle
        s_pos: Initial position of the particle
    Returns:
        BM_2d_df: DataFrame with x and y positions of the particle in pandas DataFrame
    """
    # Init velocity vector
    velocity =Vec2d(speed,0)
    
    # Init DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])    
    # Add initial position
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])    
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    
    # Generate the trajectory
    for i in range(n_steps-1):        
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
    
        temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])    
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
        
    return BM_2d_df


#####################################################################################
# Correlated random walk trajectory
#####################################################################################
def crw_2d(n_steps=1000, speed=5, coefficient=0.9, s_pos=[0,0]):
    """
    Arguments:
        n_steps: Number of steps
        speed: Speed of the particle
        coefficient: Cauchy distribution coefficient
        s_pos: Initial position of the particle
    Returns:
        CRW_2d_df: DataFrame with x and y positions of the particle in pandas DataFrame
    """
    # Init velocity vector
    velocity = Vec2d(speed, 0)

    # Init DF
    CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    # Add initial position
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
    
    # Generate the trajectory
    for i in range(n_steps-1):
        turn_angle = cauchy.rvs(scale=coefficient)
        velocity = velocity.rotated(turn_angle)
    
        temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i] + velocity.x, 'y_pos': CRW_2d_df.y_pos[i] + velocity.y}])
        CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
    
    return CRW_2d_df

#####################################################################################
# Path length for a given trajectory.
#####################################################################################
def path_length_trajectory(df):
    """
    Arguments:
        df: DataFrame with x and y positions of the particle in pandas DataFrame
    Returns:
        pl: DataFrame of path length
    """
    # Init DF
    pl = pd.DataFrame({'y_pos': [0]})

    # calculate the distance between each point
    distances = [distance.euclidean(df.iloc[i], df.iloc[i + 1]) for i in range(len(df) - 1)]
    pl = pd.DataFrame({'y_pos': np.insert(np.cumsum(distances), 0, 0)})
        
    return pl


**Compare the path length of the three trajectories Display the results using plotly.**

In [73]:
# Generate trajectories
n_steps = 1000 # Number of steps
bm_traj_3 = bm_2d(n_steps, speed=3) # Brownian motion with speed 3
bm_traj_6 = bm_2d(n_steps, speed=6) # Brownian motion with speed 6
crw_traj_6 = crw_2d(n_steps, speed=6) # Correlated random walk with speed 6

bm_pl_3 = path_length_trajectory(bm_traj_3) # Path length for BM 3
bm_pl_6 = path_length_trajectory(bm_traj_6) # Path length for BM 6
crw_pl_6 = path_length_trajectory(crw_traj_6) # Path length for CRW 6

fig = go.Figure()


# Add traces for the trajectories
fig.add_trace(go.Scatter(x=bm_pl_3.index, y=bm_pl_3.y_pos, mode='lines', name='path length BM 3'))
fig.add_trace(go.Scatter(x=bm_pl_6.index, y=bm_pl_6.y_pos, mode='lines', name='path length BM 6', line=dict(width=8)))
fig.add_trace(go.Scatter(x=crw_pl_6.index, y=crw_pl_6.y_pos, mode='lines', name='path length CRW 6'))

fig.show()



**Activity 2**: Mean Squared Displacement - (BM vs CRW) (4 pts)
- Write a function that returns the **mean squared displacement** for a given trajectory.
- Compare the **mean squared displacement** curves of at least two trajectories of
different kinds, as shown in the figure below.
- Display the results using **plotly**.

**FUNCTIONS**

In [ ]:
#####################################################################################
# Mean squared displacement for a given trajectory.
#####################################################################################
def msd(df_traj):
    """
    Arguments:
        df_traj: DataFrame with x and y positions of the trajectory in pandas DataFrame
    Returns:
        df_msd: DataFrame with mean squared displacement
    """
    # Convert DataFrame to NumPy array (for faster calculations)
    traj = df_traj[['x_pos', 'y_pos']].to_numpy().astype(float)
    
    # Init MSD array with zeros wich will be filled with values
    msd_values = np.zeros(len(traj))
    
    
    for n in range(1, len(traj)):
        displacements = traj[n:] - traj[:-n] # Calculate the displacements for each time lag
        squared_displacements = np.sum(displacements**2, axis=1) # Calculate the squared displacements
        msd_values[n] = np.mean(squared_displacements) # Calculate the mean squared displacement
    
    # Convert the result back to pandas DataFrame
    df_msd = pd.DataFrame({'y_pos': msd_values})
    
    return df_msd

Comparison of mean square displacement curves of BM 3, BM 6 and CRW 6. And results shown using plotly.

In [81]:
bm3_msd_df = msd(bm_traj_3) # MSD for BM 3
bm6_msd_df = msd(bm_traj_6) # MSD for BM 6
crw_msd_df = msd(crw_traj_6) # MSD for CRW 6, 0.9 coefficient per default

fig = go.Figure()

# Add traces for MSD of the Brownian motion 3 trajectory
fig.add_trace(go.Scatter(
    x= bm3_msd_df.index,
    y= bm3_msd_df.y_pos,
    name='MSD BM 3',
    mode='lines',
    showlegend=True
))

# Add traces for MSD of the Brownian motion 6 trajectory
fig.add_trace(go.Scatter(
    x= bm6_msd_df.index,
    y= bm6_msd_df.y_pos,
    name='MSD BM 6',
    mode='lines',
    showlegend=True
))

# Add traces for MSD of the Correlated Random Walk 6 trajectory
fig.add_trace(go.Scatter(
    x= crw_msd_df.index,
    y= crw_msd_df.y_pos,
    name='MSD CRW 6 c=0.9',
    mode='lines',
    showlegend=True
))

fig.show()